In [59]:
import requests
import time

In [60]:
def request_url(url):
    response = requests.get(url)
    while(response.status_code != 200):
        response = requests.get(url)
        time.sleep(1)
    response = response.json()
    return response

In [61]:
def get_discover_tracks_id(CLIENT_ID):
    result = set()
    
    url = f'https://api-v2.soundcloud.com/mixed-selections?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    while(1):
        response = request_url(url)
        collections = response['collection']    
        for collection in collections:
            items = collection['items']['collection']
            for item in items:
                try:
                    tracks = item['tracks']
                    for track in tracks:
                        result.add(track['id'])
                except:
                    break
        try:
            url = response['next_href']+ '&client_id='+CLIENT_ID
        except:
            break
    return result

In [62]:
CLIENT_ID='YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'

result=get_discover_tracks_id(CLIENT_ID)
len(result)

3405

In [63]:
def get_discover_playlists_id(CLIENT_ID):
    result = []
    temp=set()
    url = f'https://api-v2.soundcloud.com/mixed-selections?client_id={CLIENT_ID}&limit=100&linked_partitioning=1'
    while(1):
        response = request_url(url)
        collections = response['collection']    
        for collection in collections:
            items = collection['items']['collection']
            for item in items:
                try:
                    uri = item['uri']
                    temp.add(uri)
                except:
                    break
        try:
            url = response['next_href']+ '&client_id='+CLIENT_ID
        except:
            break
    new_temps=list(temp)
    for new_temp in new_temps:
        result.append(new_temp[37:])
    return result

In [64]:
CLIENT_ID='YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'

result = get_discover_playlists_id(CLIENT_ID)

In [65]:
def extract_playlist_generals(playlist_id, client_id):
    url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}?client_id={client_id}&limit=9&linked_partitioning=1'
    while True:
        time.sleep(1)
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        break

    print(f"Hit {response.url}")
    generals_data = response.json()
    generals_data['user'] = generals_data['user']['id']
    print(generals_data['user'])
    tracks_list = []
    for track in generals_data['tracks']:
        tracks_list.append(track['id'])
    generals_data['tracks'] = tracks_list
    return generals_data

In [66]:
def playlist_info(playlist_id, client_id):
    reposters_url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}/reposters?client_id={client_id}&limit=9&linked_partitioning=1'
    likers_url = f'https://api-v2.soundcloud.com/playlists/{playlist_id}/likers?client_id={client_id}&limit=9&linked_partitioning=1'
    generals_data = extract_playlist_generals(playlist_id, client_id)
    generals_data['reposters'] = get_id_from_collection(reposters_url, client_id, 100)
    generals_data['likers'] = get_id_from_collection(likers_url, client_id, 100)


In [67]:
def get_id_from_collection(url, client_id, result_limit):
    results = set()
    full = False
    while True:
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        print(f"Hit {response.url}")
        try:
            json_data = response.json()
            collections = json_data['collection']
            if not collections:
                break
            for collection in collections:
                if len(results) < result_limit:
                    results.add(collection['id'])
                else:
                    full = True
                    break
            if full or json_data['next_href'] is None:
                break
            url = json_data['next_href'] + f'&client_id={client_id}'
        except KeyError:
            break
    return results

In [68]:
def get_comment_id_from_collection(url, client_id, result_limit):
    results = set()
    full = False
    while True:
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        print(f"Hit {response.url}")
        try:
            json_data = response.json()
            collections = json_data['collection']
            if not collections:
                break
            for collection in collections:
                if len(results) < result_limit:
                    results.add(collection['user']['id'])
                else:
                    full = True
                    break
            if full or json_data['next_href'] is None:
                break
            url = json_data['next_href'] + f'&client_id={client_id}'
        except KeyError:
            break
    return results

In [69]:
def extract_track_generals(track_id, client_id):
    url = f'https://api-v2.soundcloud.com/tracks/{track_id}?client_id={client_id}&linked_partitioning=1'
    while True:
        time.sleep(1)
        response = requests.get(url)
        if not response.ok:
            print(f"Failed {response.url}")
            continue
        break

    print(f"Hit {response.url}")
    generals_data = response.json()
    generals_data['user'] = generals_data['user']['id']
    print(generals_data['user'])
    #tracks_list = []
    #for track in generals_data['tracks']:
    #    tracks_list.append(track['id'])
    #generals_data['tracks'] = tracks_list
    return generals_data


In [71]:
def track_info(track_id, client_id):
    reposters_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/reposters?client_id={client_id}&limit=9&linked_partitioning=1'
    likers_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/likers?client_id={client_id}&limit=9&linked_partitioning=1'
    related_tracks_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/related?client_id={client_id}&limit=10&linked_partitioning=1'
    album_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/albums?&client_id={client_id}&limit=1&linked_partitioning=1'
    playlists_url =f'https://api-v2.soundcloud.com/tracks/{track_id}/playlists_without_albums?client_id={client_id}&limit=10&linked_partitioning=1'
    comments_url = f'https://api-v2.soundcloud.com/tracks/{track_id}/comments?threaded=0&filter_replies=0&client_id={client_id}&limit=20&linked_partitioning=1'
    
    generals_data = extract_track_generals(track_id, client_id)
    generals_data['reposters'] = get_id_from_collection(reposters_url, client_id, 100)
    generals_data['likers'] = get_id_from_collection(likers_url, client_id, 100)
    generals_data['comments'] =get_comment_id_from_collection(url, client_id, 100)
    generals_data['related_tracks'] = get_id_from_collection(related_tracks_url, client_id, 100)
    generals_data['album'] = get_id_from_collection(album_url, client_id, 100)
    generals_data['playlists'] = get_id_from_collection(playlists_url, client_id, 100)
    return generals_data

In [72]:
client_id='YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79'
track_id='591945735'

generals_data=track_info(track_id, client_id)

Hit https://api-v2.soundcloud.com/tracks/591945735?client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79&linked_partitioning=1
587430921
Hit https://api-v2.soundcloud.com/tracks/591945735/reposters?client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79&limit=9&linked_partitioning=1
Failed https://api-v2.soundcloud.com/tracks/591945735/reposters?offset=1633349927000&limit=9&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Hit https://api-v2.soundcloud.com/tracks/591945735/reposters?offset=1633349927000&limit=9&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Failed https://api-v2.soundcloud.com/tracks/591945735/reposters?offset=1632344722000&limit=9&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Hit https://api-v2.soundcloud.com/tracks/591945735/reposters?offset=1632344722000&limit=9&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Hit https://api-v2.soundcloud.com/tracks/591945735/reposters?offset=1630451041000&limit=9&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Failed https://api-v2.soundcloud.com/tracks/591945735/reposters?offse

Hit https://api-v2.soundcloud.com/tracks/591945735/playlists_without_albums?offset=10&limit=10&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Hit https://api-v2.soundcloud.com/tracks/591945735/playlists_without_albums?offset=20&limit=10&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Failed https://api-v2.soundcloud.com/tracks/591945735/playlists_without_albums?offset=30&limit=10&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Failed https://api-v2.soundcloud.com/tracks/591945735/playlists_without_albums?offset=30&limit=10&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Hit https://api-v2.soundcloud.com/tracks/591945735/playlists_without_albums?offset=30&limit=10&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Hit https://api-v2.soundcloud.com/tracks/591945735/playlists_without_albums?offset=40&limit=10&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Failed https://api-v2.soundcloud.com/tracks/591945735/playlists_without_albums?offset=50&limit=10&client_id=YxQYlFPNletSMSZ4b8Svv9FTYgbNbM79
Failed https://api-v2.sou

In [75]:
def get_discover(client_id):
    
    tracks = get_discover_tracks_id(client_id)
    playlist = get_discover_playlists_id(client_id)
    
    track_result = []
    playlist_result = []
    
    for track_id in tracks:
        track_result.append(track_info(track_id, client_id))
    for playlist_id in playlists:
        playlist_result.append(playlist_info(playlist_id, client_id))
    return track_result,playlist_result